# Assignment 7

### Part A

In [8]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [9]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [10]:
def expected_return(P, rf, mkt_rp):
    
    beta = ( P['Amount Invested']*P['Beta']).sum() / P['Amount Invested'].sum()
    
    return  (mkt_rp * beta) + rf


Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [11]:
def volatility(P, cor):
    
    weight = P['Amount Invested'] / P['Amount Invested'].sum()
    vol = P['Volatility']
    
    var = ( vol[0] ** 2 * weight[0] ** 2 +
                vol[1] ** 2 * weight[1] ** 2  +
                2 * cor * vol[0] * vol[1]* weight[0] * weight[1])

    return np.sqrt(var)

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [12]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):

    market_return = mkt_rp + rf
    
    portfolio_return =  (vol / mkt_vol) * (market_return - rf) + rf 
    
    return portfolio_return

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [13]:
def CAPM_beta(return_path, factors_path):
    returns = pd.read_csv(return_path)
    factors = pd.read_csv(factors_path)

    factors['date'] = factors['date'].astype(str)
 
    factors['Mkt-RF'] = factors['Mkt-RF'] / 100
    factors['RF'] = factors['RF'] / 100 
    returns['date'] = returns['year'].astype(str) + returns['month'].astype(str).str.zfill(2)

    result = pd.merge(returns, factors, on='date')
   
    result["Excess_Mkt"] = result['Mkt-RF']
    result['Excess_RET'] = result['RET'] - result['RF']

    output = pd.DataFrame(columns=["CUSIP", "EXP_RETURN", "BETA"])

    for cusip, group in result.groupby('CUSIP'):
        cov_matrix = np.cov(group['Excess_RET'], group['Excess_Mkt'], ddof=0)
        beta = cov_matrix[0, 1] / cov_matrix[1, 1]
        exp_return = group['RET'].mean()
        output = pd.concat([output, pd.DataFrame({"CUSIP": [cusip], "EXP_RETURN": [exp_return], "BETA": [beta]})], ignore_index=True)

    return output





Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [14]:
def slope(data):
    
    a = data['BETA']
    b = data['EXP_RETURN']

    result_1,result_2 = np.polyfit(a, b, 1)
    return result_2


   # This should return 0.007981588120535642... with the provided data.     # This should return 0.007981588120535642... with the provided data.   # This should return 0.007981588120535642... with the provided data.